This tutorial uses the same example as the introduction. In other words:

In [1]:
# Module we will use
import chaospy
import numpy
from matplotlib import pyplot

def model_predictor(coordinates, parameters):
    """Over simplistic model function."""
    param_I, param_a = parameters
    return param_I*numpy.e**(-param_a*coordinates)

# The spatio-temporal coordinates 
coordinates = numpy.linspace(0, 10, 1000)

# The distribution of the parameters
distribution_I = chaospy.Normal(1.5, 0.2)
distribution_a = chaospy.Uniform(0.1, 0.2)
distribution = chaospy.J(distribution_I, distribution_a)

## Point Collocation Method (PCM)

Point collection method is a broad term, as it coveres multiple approaches.
But in a nutshell:

* Generate $Q_1, ..., Q_N = (I_1, a_1), ..., (I_N, a_N)$, using (psuedo-)random samples or otherwise.
* Evaluate model predictor $U_1, ..., U_N$ for each sample.
* Select an expansion or polynomials $P_1, ..., P_M$, typically orthogonal (with respect to the probability distribution).
* Define the linear regression problem: $U_n = \sum_m c_m(t) P_m(I_n, a_n)$ and solve for $c_1, ..., c_M$.

This approach does not require the samples to follow any distribution, nor does it require the polynomial expansion to have any particular properties, like e.g. orthogonality.
In practice however, when focusing on statistical properties (like mean and variance), it makes sense to have the number of samples in an area roughly proportional to the probability density function.
This can be achieved in a few ways, but most common is to either use random samples/low deiscrepency sequences, or abscissas from a quarature scheme.
And in the same way, even though the polynomials do not have to be orthogonal, in practice using orthogonal polynomials are shown to be more numerical stable.

Below, we will compare the [Halton sequence](https://en.wikipedia.org/wiki/Halton_sequence) and compare it to [Leja quadrature](https://arxiv.org/pdf/1404.5663.pdf).
(The reason for the latter, is that Leja is both perfectly nested, making error analysis computaitonal cheapter, and a all around scheme working well for both Normal and Uniform distribution.)

### Generating samples

In chaospy, creating low-discrepency sequences can be done using the `distribution.sample` method.
Creating quadrature points can be done using `chaospy.generate_quadrature`.

In [5]:
# Sampling schemes we want to include
rules = ["halton", "leja"]

# Generate samples for Leja for different orders:
samples = {"leja": [chaospy.generate_quadrature(order, distribution, rule="leja") for order in range(2, 4)]}
samples

UnboundLocalError: local variable 'fu' referenced before assignment